In [1]:
import camelot
import matplotlib.pyplot as plt
import pandas as pd
import re
import datetime
import PyPDF2

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
filename="A:/Jerome/Python Project/SamplePDFs/2102 Lines.pdf"

In [4]:
pdf=open(filename,'rb')
startpg=2
lastpg=PyPDF2.PdfFileReader(pdf).getNumPages()
beforelastpg=lastpg-1
pgrange=str(startpg)+'-'+str(beforelastpg)
pdf.close()

In [6]:
dflist1=[]
dflist2=[]

In [5]:
tables=[]

In [7]:
while True:
        try:
            extract=camelot.read_pdf(filename,flavor='stream',pages=pgrange,table_areas=['79.6,468,779.7,384.8','42.4,466,80.5,314.5','79.6,313,779.7,230.98','79.6,384.8,779.7,314.0','79.6,230.98,779.7,159.0','42.4,314,80.5,159.0'],columns=['104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','42.4','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','42.4'],strip_text='--\n')
            for i in range(0,len(extract)):
                tables.append(extract[i].df)
            break
        except:
            startpg=startpg+1
            pgrange=str(startpg)+'-'+str(beforelastpg)
            print("  Updated page range:",pgrange)

  Updated page range: 2-82


In [8]:
try:
        pgrange=str(lastpg)
        extractlast=camelot.read_pdf(filename,flavor='stream',pages=pgrange,table_areas=['79.6,468,779.7,384.8','42.4,466,80.5,314.5','79.6,313,779.7,230.98','79.6,384.8,779.7,314.0','79.6,230.98,779.7,159.0','42.4,314,80.5,159.0'],columns=['104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','42.4','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','42.4'],strip_text='--\n')
        for i in range(0,len(extractlast)):
                tables.append(extractlast[i].df)
except:
    
    extractlast=camelot.read_pdf(filename,flavor='stream',pages=pgrange,table_areas=['79.6,468,779.7,384.8','42.4,466,80.5,314.5','79.6,384.8,779.7,314.0'],columns=['104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3','42.4','104.6,128.8,152.9,177.1,201.2,225,249.5,273.4,297.6,321.6,345.6,369.8,393.9,418.2,442.1,466.4,490.4,514.3,538.6,562.7,586.8,611,635,659.1,683.4,707.3,731.3'],strip_text='--\n')
    for i in range(0,len(extractlast)):
                tables.append(extractlast[i].df)

In [9]:
for i in range(0,int(len(tables)),6):
        
        no_com_table=tables[i].loc[1:].reset_index(drop=True)
        
        if no_com_table.iloc[0,0] in ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"] and tables[i+2].iloc[0,0] in ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]:
            df=pd.concat([tables[i].loc[1:].reset_index(drop=True),tables[i+2]],axis=1,ignore_index=True)
            dflist1.append(df)
            try:
                df=pd.concat([tables[i+3].loc[1:].reset_index(drop=True),tables[i+4]],axis=1,ignore_index=True)
                dflist1.append(df)
            except:
                break

In [10]:
for i in range (0,len(dflist1)):   
        
        if len(dflist1[i].index)==5:
            empty_row=pd.DataFrame({}, index=[2.5])
            dflist1[i]=pd.concat([dflist1[i].loc[:2],empty_row, dflist1[i].loc[3:]]).reset_index(drop=True)
            dflist1[i]=dflist1[i].fillna(' ')
            
        dflist1[i].rename(index={0:"Day",1:"Date",2:"trip #",3:"Destination",4:"Time",5:"# Hours"},inplace=True)
print("  Lines obtained")

  Lines obtained


In [11]:
len(dflist1)

163

In [12]:
dflist1[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
Day,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat
Date,24,25,26,27,28,29,30,31,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
trip #,,,10,10,10,,,,,12,10,10,15,,,,10,10,10,15,,,,10,,,,,,,10,10,10,6,38,,,,,,,41,,,12,10,10,15,45,,,,,,,58
Destination,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Time,,,0746,0746,0746,,,,,0825,0746,0746,0737,,,,0746,0746,0746,0737,,,,0746,,,,,,,0746,0746,0746,0713,0852,,,,,,,0852,,,0825,0746,0746,0737,0917,,,,,,,0732
# Hours,,,6:00,6:00,6:00,,,,,6:00,6:00,6:00,6:00,,,,6:00,6:00,6:00,6:00,,,,6:00,,,,,,,6:00,6:00,6:00,6:02,6:00,,,,,,,6:00,,,6:00,6:00,6:00,6:00,6:00,,,,,,,6:00


In [13]:
dflist1[162]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
Day,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat
Date,24,25,26,27,28,29,30,31,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
trip #,,RC,RC,RC,RC,RC,,,RC,RC,RC,RC,RC,,,,RC,RC,RC,RC,,,,,,,,,,RC,RC,RC,RC,RC,,,RC,RC,RC,RC,RC,,,,RC,RC,RC,RC,,,,,,,,


In [14]:
tables[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,Comment:,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat
2,24,25,26,27,28,29,30,31,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
3,,,10,10,10,,,,,12,10,10,15,,,,10,10,10,15,,,,10,,,,
4,,,0746,0746,0746,,,,,0825,0746,0746,0737,,,,0746,0746,0746,0737,,,,0746,,,,
5,,,6:00,6:00,6:00,,,,,6:00,6:00,6:00,6:00,,,,6:00,6:00,6:00,6:00,,,,6:00,,,,


In [15]:
tables[1]

,0,1
0,,SDF 1
1,,1/1/0/
2,,PP1 (2102)
3,,CT: 72:00
4,,BT: 52:13
5,,DO: 12
6,,DD: 12
7,,PP2 (2103)
8,,CT: 72:02
9,,BT: 38:37


In [16]:
tables[2]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat
1,21,22,23,24,25,26,27,28,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
2,,,10,10,10,6,38,,,,,,,41,,,12,10,10,15,45,,,,,,,58
3,,,0746,0746,0746,0713,0852,,,,,,,0852,,,0825,0746,0746,0737,0917,,,,,,,0732
4,,,6:00,6:00,6:00,6:02,6:00,,,,,,,6:00,,,6:00,6:00,6:00,6:00,6:00,,,,,,,6:00


In [17]:
tables[3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,Comment:,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat
2,24,25,26,27,28,29,30,31,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
3,,,9,9,9,9,,,,9,9,9,9,,,,9,9,9,9,,,,,,,,
4,,,0745,0745,0745,0745,,,,0745,0745,0745,0745,,,,0745,0745,0745,0745,,,,,,,,
5,,,6:00,6:00,6:00,6:00,,,,6:00,6:00,6:00,6:00,,,,6:00,6:00,6:00,6:00,,,,,,,,


In [18]:
tables[4]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Sat
1,21,22,23,24,25,26,27,28,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
2,,,9,9,9,9,,,,9,9,9,9,,,,,,,,,,,,49,49,49,60
3,,,0745,0745,0745,0745,,,,0745,0745,0745,0745,,,,,,,,,,,,0645,0645,0645,0746
4,,,6:00,6:00,6:00,6:00,,,,6:00,6:00,6:00,6:00,,,,,,,,,,,,6:00,6:00,6:00,6:25


In [19]:
tables[5]

,0,1
0,,SDF 2
1,,1/1/0/
2,,PP1 (2102)
3,,CT: 72:00
4,,BT: 48:00
5,,DO: 12
6,,DD: 12
7,,PP2 (2103)
8,,CT: 72:25
9,,BT: 45:26


In [33]:
n=0
dflist2=[]
m=0

In [28]:

for i in range(0,len(tables),6):
    
    emptydf=pd.DataFrame(columns=["Line #","CT","Comment"])
    if i+5<len(tables):
        for (index1, row1),(index2,row2) in zip(tables[i+1].iterrows(),tables[i+5].iterrows()):
        
            row_str=' '.join(row1.astype(str))
            SDF=re.search(r"SDF  (\d+)",row_str)
            CT=re.search(r"CT: (\d{2}):(\d{2})",row_str)
            Comment=re.search(r'Comment:(.*)',str(tables[i].loc[0,0]))
            print(SDF)
            if SDF:
                dflist2.append(emptydf)
                dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
                dflist2[n].loc[0,"Comment"]=str(Comment.group(1))
            
            elif CT:
                h=float(CT.group(1))
                min=float(CT.group(2))
                CT=h + min/60
                if index1>5:
                    CT2= CT
                    dflist2[n].loc[0,"CT"]= CT1+CT2
                    n+=1
                    
                CT1=CT
        
            row_str2=' '.join(row2.astype(str))
            SDF2=re.search(r"SDF  (\d+)",row_str2)
            CT2=re.search(r"CT: (\d{2}):(\d{2})",row_str2)
            Comment=re.search(r'Comment:(.*)',str(tables[i+3].loc[0,0]))
            print(SDF2)
            if SDF2:
                dflist2.append(emptydf)
                dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
                dflist2[n].loc[0,"Comment"]=str(Comment.group(1))               
            
            elif CT2:
                h=float(CT2.group(1))
                min=float(CT2.group(2))
                CT=h + min/60
                if index1>5:
                    CT22= CT
                    dflist2[n].loc[0,"CT"]= CT1+CT22
                    n+=1
                CT1=CT
            

    else:
        for index1, row1 in tables[i+1].iterrows():
            
            row_str=' '.join(row1.astype(str))
            SDF=re.search(r"SDF  (\d+)",row_str)
            CT=re.search(r"CT: (\d{2}):(\d{2})",row_str)
            Comment=re.search(r'Comment:(.*)',str(tables[i].loc[0,0]))
            if SDF:
                dflist2.append(pd.DataFrame(columns=["Line #","CT","Comment"]))
                dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
                dflist2[n].loc[0,"Comment"]=str(Comment.group(1))
                print(dflist2[n].loc[0,"Line #"])
            elif CT:
                h=float(CT.group(1))
                min=float(CT.group(2))
                CT=h + min/60
                if index1>5:
                    CT2= CT
                    dflist2[n].loc[0,"CT"]= CT1+CT2
                    n+=1
                CT1=CT
         
        

<re.Match object; span=(1, 7), match='SDF  1'>
<re.Match object; span=(1, 7), match='SDF  2'>
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(1, 7), match='SDF  3'>
<re.Match object; span=(1, 7), match='SDF  4'>
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(1, 7), match='SDF  5'>
<re.Match object; span=(1, 7), match='SDF  6'>
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(1, 7), match='SDF  7'>
<re.Match object; span=(1, 7), match='SDF  8'>
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(1, 7), match='SDF  9'>
<re.Match object; span=(1, 8), match='SDF  10'>
None
None
None
None
None
None
None
None
None
None

In [30]:
dflist2[1]

,Line #,CT,Comment
0,1,144.45,


In [41]:
for i in range(0,len(tables),6):
   
    emptydf=pd.DataFrame(columns=["Line #","CT","Comment"])
    for index1, row1 in tables[i+1].iterrows():
        
        row_str=' '.join(row1.astype(str))
        SDF=re.search(r"SDF  (\d+)",row_str)
        CT=re.search(r"CT: (\d{2}):(\d{2})",row_str)
        Comment=re.search(r'Comment:(.*)',str(tables[i].loc[0,0]))
            
        if SDF:
                
            dflist2.append(emptydf)
            dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
            dflist2[n].loc[0,"Comment"]=str(Comment.group(1))
            
        elif CT:
            h=float(CT.group(1))
            min=float(CT.group(2))
            CT=h + min/60
            if index1>5:
                CT2= CT
                dflist2[n].loc[0,"CT"]= CT1+CT2
                print(dflist2[n])
                n+=1
                    
            CT1=CT
    if i+5>len(tables):
        for index2,row2 in tables[i+5].iterrows():
            row_str=' '.join(row2.astype(str))
            SDF2=re.search(r"SDF  (\d+)",row_str)
            CT=re.search(r"CT: (\d{2}):(\d{2})",row_str)
            Comment=re.search(r'Comment:(.*)',str(tables[i+3].loc[0,0]))
            if SDF2:
                dflist2.append(emptydf)
                dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
                dflist2[n].loc[0,"Comment"]=str(Comment.group(1))
                
            
            elif CT:
                h=float(CT.group(1))
                min=float(CT.group(2))
                CT=h + min/60
                if index1>5:
                    CT2= CT
                    dflist2[n].loc[0,"CT"]= CT1+CT2
                    
                    n+=1
                CT1=CT
    else:
        break

  Line #          CT Comment
0      1  144.033333        


In [34]:
for i in range(0,len(tables),6):
    
    emptydf=pd.DataFrame(columns=["Line #","CT","Comment"])
    if i+5<len(tables):
        for (index1, row1),(index2,row2) in zip(tables[i+1].iterrows(),tables[i+5].iterrows()):
            row_str=' '.join(row1.astype(str))
            SDF=re.search(r"SDF  (\d+)",row_str)
            CT=re.search(r"CT: (\d{2}):(\d{2})",row_str)
            Comment=re.search(r'Comment:(.*)',str(tables[i+3].loc[0,0]))
            row_str2=' '.join(row2.astype(str))
            SDF2=re.search(r"SDF  (\d+)",row_str2)
            CT2=re.search(r"CT: (\d{2}):(\d{2})",row_str2)
            
            if SDF:
                dflist2.append(emptydf)
                dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
                dflist2[n].loc[0,"Comment"]=str(Comment.group(1))
                n+=1
            elif SDF2:
                dflist2.append(emptydf)
                dflist2[n].loc[0,"Line #"]=int(SDF.group(1))
                dflist2[n].loc[0,"Comment"]=str(Comment.group(1))
                n+=1
            
            if CT:
                h=float(CT.group(1))
                min=float(CT.group(2))
                CT=h + min/60
                if index1>5:
                    CT2= CT
                    dflist2[n].loc[0,"CT"]= CT1+CT2
                    
                    
                CT1=CT
            elif CT2:
                h=float(CT2.group(1))
                min=float(CT2.group(2))
                CT=h + min/60
                if index1>5:
                    CTtwo= CT
                    dflist2[n].loc[0,"CT"]= CT1+CTtwo
                    
                CT1=CT
            

IndexError: list index out of range